# 使用Neo4j和LangChain集成非结构化和图知识增强煤矿事故QA （CoalMineLLM-InternLM2-Chat-1_8B版）

##  1 环境依赖安装

In [1]:
%pip install langchain openai tiktoken neo4j transformers python-dotenv langchain_openai

Note: you may need to restart the kernel to use updated packages.


## 2 导包

In [3]:
import os
import re
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

os.environ["OPENAI_API_KEY"] = "sk-..." #填入你自己的OpenAI API key
os.environ["OPENAI_API_BASE"] = "https://api.xiaoai.plus/v1"  #调整API请求地址，设置访问中转代理服务器，如果商家购买的，可以联系商家要代理服务器地址，这里并不是固定的
os.environ["NEO4J_URI"] = "neo4j://localhost:7687" # Neo4j数据库的URI
os.environ["NEO4J_USERNAME"] = "neo4j" #设置Neo4j数据库的用户名
os.environ["NEO4J_PASSWORD"] = "neo4j"#设置Neo4j数据库的密码

## 3 读取本地事故报告文本

In [4]:
with open(r'C:\Users\jys\OneDrive\桌面\accidents.txt',encoding='utf-8') as f:
    accidents_data_txt = f.read()

## 4 加载bert分词器

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

def bert_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

D:\anaconda\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jys\.cache\huggingface\hub\models--bert-base-chinese. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

##  5 切块

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
          chunk_size = 1000,  # 切块大小
          chunk_overlap  = 200,  # 切块重叠大小
          length_function = bert_len,
          separators = ["\n",' '],
      )
documents = text_splitter.create_documents([accidents_data_txt])

Token indices sequence length is longer than the specified maximum sequence length for this model (650 > 512). Running this sequence through the model will result in indexing errors


In [7]:
#打印分块chunk个数
print(len(documents))

11


## 6 把切块好的文本块集合存入到Neo4j向量数据库中

In [8]:
neo4j_vector = Neo4jVector.from_documents(
    documents,
    OpenAIEmbeddings(),
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"]
)

D:\anaconda\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## 7 检索query相关的Top2文本块

In [14]:
query = "山西兰花集团营山煤矿有限公司发生了什么事故？"
vector_results = neo4j_vector.similarity_search(query, k=2)
for i, res in enumerate(vector_results):
    print("第{}个文本块\n".format(i+1))
    print(res.page_content)
    print("\n")
   
#返回
vector_result = vector_results[0].page_content + vector_results[1].page_content

第1个文本块

日期和时间: 2022年11月22日15时50分
地点: 崇信县周寨煤业有限责任公司措施斜井
事故类别: 运输事故
死亡人数: 1人
受伤人数: 0人
经济损失: 158.227万元

4.2023年1月27日4时38分许，山西兰花集团营山煤矿有限公司ZF1212进工作面发生水害事故，造成4人死亡，事故直接经济损失876.12万元。
事故直接原因:经综合分析认定: 山煤矿在未消除复采区域水害事故隐患 的情况下安排掘进 ZF1212 巷，超前钻探(长探)违反《煤矿安全规程》，逐班探测(短探)违反作业规程，前方积水未探明，危险掘进，受老空水的长期浸泡和放炮对围岩(煤)的破坏并叠加采场应力与水压力耦合的影响，3#煤上部老空积水突破厚度 有限的煤壁瞬间溃入工作面，造成 4 人遇难，是本起事故的直接原因。
事故间接原因:
(1)山煤矿重生产，轻安全。违反兰花科创《综采放顶煤工作面管理办法》规定，在 ZF121 放顶煤工作面开采设计未批复，周 边采空积水未疏放干净、隐患未消除的情况下提前开掘 ZF1211 巷、ZF1212 巷。
(2) 山煤矿防治水工作责任层层悬空，管理混乱。违反《煤矿防治水细则》第四十八条第(一)项规定，擅自将探放水钻孔超 前距确定为 10m;防治水制度不落实，探水钻孔钻探、验收、移交 工作未按要求执行;未认真进行物探，物探报告造假，探放水弄虚作假;长探不到位、短探不落实
(3)山煤矿隐患排查流于形式，现场安全监督形同虚设。未按《中华人民共和国安全生产法》第四十一条规定，对辨识出的 水害重大风险未采取相应管控措施，安全隐患大排查流于形式，现 场安全管理混乱。工业视频未有效利用，探放水相关制度及作业规程在现场得不到有效落实，现场安全监督形同虚设
(4) 兰花集团企业主体责任不落实，疏于管理。2022 年7月1日将日常安全管理责任委托给兰花科创后，日常疏于安全监督管理，总工程师对营山煤矿的复采防治水工作未引起足够重视，包矿领导到矿检查期间也未发现营山煤矿防治水工作造假的情况。
(5) 兰花科创安全管理不到位，日常安全监督检查不严不细。自 2022 年 7月 1 日受委托对营山煤矿进行安全监管后，安全管理不 到位，日安全监督检查不认真、不仔细，未发现 ZF121 放顶煤工作面未批先掘以及探放水方面存在的诸多问题。


第2个文本块

3.矿井隐患排

In [15]:
print(vector_result)  #作为非结构化知识信息

日期和时间: 2022年11月22日15时50分
地点: 崇信县周寨煤业有限责任公司措施斜井
事故类别: 运输事故
死亡人数: 1人
受伤人数: 0人
经济损失: 158.227万元

4.2023年1月27日4时38分许，山西兰花集团营山煤矿有限公司ZF1212进工作面发生水害事故，造成4人死亡，事故直接经济损失876.12万元。
事故直接原因:经综合分析认定: 山煤矿在未消除复采区域水害事故隐患 的情况下安排掘进 ZF1212 巷，超前钻探(长探)违反《煤矿安全规程》，逐班探测(短探)违反作业规程，前方积水未探明，危险掘进，受老空水的长期浸泡和放炮对围岩(煤)的破坏并叠加采场应力与水压力耦合的影响，3#煤上部老空积水突破厚度 有限的煤壁瞬间溃入工作面，造成 4 人遇难，是本起事故的直接原因。
事故间接原因:
(1)山煤矿重生产，轻安全。违反兰花科创《综采放顶煤工作面管理办法》规定，在 ZF121 放顶煤工作面开采设计未批复，周 边采空积水未疏放干净、隐患未消除的情况下提前开掘 ZF1211 巷、ZF1212 巷。
(2) 山煤矿防治水工作责任层层悬空，管理混乱。违反《煤矿防治水细则》第四十八条第(一)项规定，擅自将探放水钻孔超 前距确定为 10m;防治水制度不落实，探水钻孔钻探、验收、移交 工作未按要求执行;未认真进行物探，物探报告造假，探放水弄虚作假;长探不到位、短探不落实
(3)山煤矿隐患排查流于形式，现场安全监督形同虚设。未按《中华人民共和国安全生产法》第四十一条规定，对辨识出的 水害重大风险未采取相应管控措施，安全隐患大排查流于形式，现 场安全管理混乱。工业视频未有效利用，探放水相关制度及作业规程在现场得不到有效落实，现场安全监督形同虚设
(4) 兰花集团企业主体责任不落实，疏于管理。2022 年7月1日将日常安全管理责任委托给兰花科创后，日常疏于安全监督管理，总工程师对营山煤矿的复采防治水工作未引起足够重视，包矿领导到矿检查期间也未发现营山煤矿防治水工作造假的情况。
(5) 兰花科创安全管理不到位，日常安全监督检查不严不细。自 2022 年 7月 1 日受委托对营山煤矿进行安全监管后，安全管理不 到位，日安全监督检查不认真、不仔细，未发现 ZF121 放顶煤工作面未批先掘以及探放水方面存在的诸多问题。3.矿井隐患排查治理不到位。企业隐患排查过程中，未排

## 8 构建Neo4j QA chain

In [16]:
graph = Neo4jGraph(
    url=os.environ["NEO4J_URI"], username=os.environ["NEO4J_USERNAME"], password=os.environ["NEO4J_PASSWORD"]
)

In [17]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

D:\anaconda\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## 9 Neo4j图知识查询

In [20]:
graph_result = chain.run("山西兰花集团营山煤矿有限公司发生了什么事故？")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:煤矿名称{name:"山西兰花集团营山煤矿有限公司"})-[:发生事故类别]->(事故类别) RETURN 事故类别.name
Full Context:
[{'事故类别.name': '水害事故'}]

> Finished chain.


In [21]:
graph_result

'水害事故。'

## 10 使用CoalMineLLM-InternLM2-Chat-1_8B模型，进行煤矿事故QA

In [23]:
query = "山西兰花集团营山煤矿有限公司发生了什么事故？"
final_prompt = f"""你是一个乐于助人的问答代理人。你的任务是分析并综合来自两个来源的信息：相似性搜索top2的结果数据
（非结构化信息）和图数据库查询出的相关数据（结构化信息）。
给定用户的查询：{query},准确且高效地回答用户提出问题。
从以下数据中提供回答的上下文:
非结构化信息: {vector_result}.
结构化信息: {graph_result}.
"""

In [27]:
!pip install einops

   ---------------------------------------- 43.2/43.2 kB 95.9 kB/s eta 0:00:00


In [31]:
from transformers import AutoTokenizer, AutoModel, GenerationConfig, AutoModelForCausalLM
model_path =r"C:\Users\jys\Downloads\CoalMineLLM_InternlM_1_8B-Chat"
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = model.eval()
response, history = model.chat(tokenizer, final_prompt, history=[])
response

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

'根据提供的信息，2022年11月22日15时50分，山西兰花集团营山煤矿有限公司发生了一起水害事故，造成4人死亡，直接经济损失为876.12万元。事故直接原因是未消除复采区域水害隐患，前方积水未探明，危险掘进，受老空水长期浸泡和放炮对围岩（煤）造成破坏并叠加采场应力与水压力耦合的影响，积水突破了煤壁瞬间溃入工作面。事故间接原因包括违反安全规程、水害隐患未消除、现场掘进作业、防治水工作责任悬空、隐患排查流于形式、主体责任不落实、安全管理不到位和职工安全培训教育不到位。'